In [53]:

# A Simple RAG Pipeline
import os
from dotenv import load_dotenv, dotenv_values
from utils import clean_text, print_chunks, rerank_chunks, reset_pinecone, embed_and_upsert, get_relevant_chunks

# Load All ENV Values
load_dotenv()
pinecone_namespace="simple-rag-2-he"

file_path="docs/ת''ע (פ''ת) 20945-05-22 ג' ואח' נ' האפוטרופוס הכללי במחוז ת''א ואח' (נבו 14.3.2024) _ עו''ד עקיבא גולן.md"
# 1. Load the txt file
with open(file_path, 'r', encoding='utf-8') as file:
    # Read the content of the file
    text = file.read()

text = text.replace("\\n", "\n").replace("\\", "").replace(":unselected:", "");
print(text)

<!-- PageHeader="תע (פ״ת) 20945-05-22" -->

<!-- PageHeader="ר׳ ג׳ נ׳ י׳ ג׳ (אדם שמונה לו אפוטרופוס)" -->

בית משפט לענייני משפחה בפתח תקווה
===

ת״ע 20945-05-22 ג׳ ואח׳ נ׳ האפוטרופוס הכללי במחוז תל-אביב ואח׳ ת״ע 20952-05-22 ת״ע 20948-05-22 ת״ע 20954-05-22

תלה״מ 43870-12-19

תיק חיצוני: 324757_3

כבוד השופטת שירלי שי

לפני

תובעים/נתבעים

1. ר׳ ג׳ 2. ב׳ ג׳

ע״י ב״כ עוה״ד עקיבא גולן ו/או עו״ד יהודה יוד

נגד

י׳ ג׳ (אדם שמונה לו אפוטרופוס) באמצעות מיופת כוחו ב׳ ג׳ ע״י ב״כ עוה״ד עופר גרינפלד ו/או עו״ד הילה צאירי

נתבע/תובע:

חקיקה שאוזכרה:

חוק הירושה, תשכ״ה-1965: סע' '8, ומ 8 א, 26, 30, 30 (א), 35


# פסק דין

1. פסק דין יעסוק ב- 5 הליכים הקשורים כולם לעזבונה של הגב׳ ר׳ ג׳ ז״ל (להלן: "המנוחה") שהלכה לבית עולמה ביום 10.9.2021 כלדלהלן:

א. ת״ע 20945-05-22 - בקשה למתן צו לקיום צוואת המנוחה מיום 25.2.2019 (להלן "הצוואה השנייה"), אשר הוגשה על ידי שני בניה של המנוחה (להלן: יקראו יחדיו "הבנים" ובהתאמה ר׳ ו/או ב').

ב. ת״ע 20952-05-22 - התנגדות לבקשת הבנים לקיום הצוואה השנייה, שהוגשה על ידי מי 

In [54]:
print("----- Cleaned Text -----")
print(clean_text(text))

----- Cleaned Text -----
<!-- PageHeader="תע (פ״ת) 20945-05-22" -->
<!-- PageHeader="ר׳ ג׳ נ׳ י׳ ג׳ (אדם שמונה לו אפוטרופוס)" -->
בית משפט לענייני משפחה בפתח תקווה
===
ת״ע 20945-05-22 ג׳ ואח׳ נ׳ האפוטרופוס הכללי במחוז תל-אביב ואח׳ ת״ע 20952-05-22 ת״ע 20948-05-22 ת״ע 20954-05-22
תלה״מ 43870-12-19
תיק חיצוני: 324757_3
כבוד השופטת שירלי שי
לפני
תובעים/נתבעים
1. ר׳ ג׳ 2. ב׳ ג׳
ע״י ב״כ עוה״ד עקיבא גולן ו/או עו״ד יהודה יוד
נגד
י׳ ג׳ (אדם שמונה לו אפוטרופוס) באמצעות מיופת כוחו ב׳ ג׳ ע״י ב״כ עוה״ד עופר גרינפלד ו/או עו״ד הילה צאירי
נתבע/תובע:
חקיקה שאוזכרה:
חוק הירושה, תשכ״ה-1965: סע' '8, ומ 8 א, 26, 30, 30 (א), 35
# פסק דין
1. פסק דין יעסוק ב- 5 הליכים הקשורים כולם לעזבונה של הגב׳ ר׳ ג׳ ז״ל (להלן: "המנוחה") שהלכה לבית עולמה ביום 10.9.2021 כלדלהלן:
א. ת״ע 20945-05-22 - בקשה למתן צו לקיום צוואת המנוחה מיום 25.2.2019 (להלן "הצוואה השנייה"), אשר הוגשה על ידי שני בניה של המנוחה (להלן: יקראו יחדיו "הבנים" ובהתאמה ר׳ ו/או ב').
ב. ת״ע 20952-05-22 - התנגדות לבקשת הבנים לקיום הצוואה השנייה, שהוגשה על יד

In [55]:
# 2. Create Chunks

from langchain_text_splitters import MarkdownHeaderTextSplitter
text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[("#", "h1"), ("##", "h2"), ("###", "h3")], strip_headers=False)
splits = text_splitter.split_text(clean_text(text))
# Create chunks, extract text
texts = list(map((lambda x: x.page_content), splits))
print_chunks(texts)

Total Chunks:  16
--------
CHUNK 1
--------
<!-- PageHeader="תע (פ״ת) 20945-05-22" -->
<!-- PageHeader="ר׳ ג׳ נ׳ י׳ ג׳ (אדם שמונה לו אפוטרופוס)" -->
בית משפט לענייני משפחה בפתח תקווה
===
ת״ע 20945-05-22 ג׳ ואח׳ נ׳ האפוטרופוס הכללי במחוז תל-אביב ואח׳ ת״ע 20952-05-22 ת״ע 20948-05-22 ת״ע 20954-05-22
תלה״מ 43870-12-19
תיק חיצוני: 324757_3
כבוד השופטת שירלי שי
לפני
תובעים/נתבעים
1. ר׳ ג׳ 2. ב׳ ג׳
ע״י ב״כ עוה״ד עקיבא גולן ו/או עו״ד יהודה יוד
נגד
י׳ ג׳ (אדם שמונה לו אפוטרופוס) באמצעות מיופת כוחו ב׳ ג׳ ע״י ב״כ עוה״ד עופר גרינפלד ו/או עו״ד הילה צאירי
נתבע/תובע:
חקיקה שאוזכרה:
חוק הירושה, תשכ״ה-1965: סע' '8, ומ 8 א, 26, 30, 30 (א), 35
--------
CHUNK 2
--------
# פסק דין
1. פסק דין יעסוק ב- 5 הליכים הקשורים כולם לעזבונה של הגב׳ ר׳ ג׳ ז״ל (להלן: "המנוחה") שהלכה לבית עולמה ביום 10.9.2021 כלדלהלן:
א. ת״ע 20945-05-22 - בקשה למתן צו לקיום צוואת המנוחה מיום 25.2.2019 (להלן "הצוואה השנייה"), אשר הוגשה על ידי שני בניה של המנוחה (להלן: יקראו יחדיו "הבנים" ובהתאמה ר׳ ו/או ב').
ב. ת״ע 20952-05-22 - התנגדות 

In [56]:
# 3. Reset Pinecone and Upsert Chunks. Only Run Once
# reset_pinecone(pinecone_namespace)
embed_and_upsert(texts, pinecone_namespace)

True

In [38]:
from utils import get_relevant_chunks
from search_queries import kareem_01_queries

# https://chatgpt.com/share/271c8eea-0efc-4c6d-9e76-391a11eecc1b
queries = kareem_01_queries
query = queries[3]["hebrew"]
relevant_chunks = get_relevant_chunks(query, pinecone_namespace, 10)
relevant_chunks_text = [item["metadata"]["text"] for item in relevant_chunks]

print(pinecone_namespace)
print(query)
print_chunks(relevant_chunks, True)

simple-rag-2-he
מה הסיבות לאי תחולת חוק הנאמנות על סוגי נאמנויות מסוימות?
Total Chunks:  10
--------
CHUNK 1
--------
Id: 7, Score: 0.915123284
## ג. הדין בישראל לפני חוק הנאמנות
--------
CHUNK 2
--------
Id: 34, Score: 0.91433084
## ט. מי רשאי לדרוש אכיפת הנאמנות?
--------
CHUNK 3
--------
Id: 32, Score: 0.912677348
## 2. החובה להחזיק בנפרד את נכסי הנאמנות
225 אמצעי לזיהוי נכסי הנאמנות הם הפרדה פיזית ביניהם לבין נכסי הנאמן, ככל האפשר, 268 ומתן הודעה על קיום נאמנות, לשם רישומה בפנקס-הזכויות שבו רשומים נכסי הנאמנות. 269 -
הפרדה זו יש בה כדי לאפשר בדיקה אם הנאמן שלח את ידו בנכסי הנאמנות, או שמא הוא מזניח את השמירה עליהם.
|||
| - | - |
| סעיף 10 (ג) לחוק הנאמנות. | 267 |
| סעיף 3 (ג) לחוק הנאמנות. | 268 |
| סעיף 4 לחוק הנאמנות. | 269 |
76
<!-- PageFooter="nero. co.il" -->
<!-- PageNumber="§23" -->
<!-- PageHeader="חובות להבטחת אמצעי פיקוח על הנאמן" -->
מבוא
--------
CHUNK 4
--------
Id: 35, Score: 0.908864498
## 1. יחס הנהנה - נאמן
245 חוק הנאמנות חל על הסדרים משפטיים שונים. חלק מהסדרים א

In [39]:
# Lets use a reranker to see how much it helps
rerank_results = rerank_chunks(query, relevant_chunks_text)
print_chunks(rerank_results, True)

Total Chunks:  10
--------
CHUNK 1
--------
Id: 5, Score: 0.69921875
## יג. פרשנות החוק
295 בדרך-כלל ניתן להסתייע בפרשנות חוק בסיסי, דוגמת חוק הנאמנות, בעבודות ההכנה שקדמו לחוק, ובמיוחד בדברי ההסבר להצעת החוק. אולם במקרים רבים לא ניתן היה להסתייע בדברי ההסבר להצעת החוק, גם כשלכאורה ניסוח החוק דומה לנוסח הצעת החוק, כיוון שהחוק שונה מההצעה בתפישותיו היסודיות וכן בפרטים מהותיים.
:
:
-
--
i
.
..
306 סעיף 83.
307 סעיף 72.
89
<!-- PageFooter="nero.co.il" -->
<!-- PageNumber="§29" -->
<!-- PageHeader="מבוא" -->
<!-- PageHeader="פרשנות החוק" -->
בפרשנות החוק הנחנו, שיש לראות את הוראות חוק הנאמנות כמשתלבות, ולכן כמתיישבות, עם הוראות חוקים אחרים, העוסקות באותם נושאים או בנושאים קרובים. הנחה זו מסתברת מסעיף 42 לחוק, המכיר בעדיפותו של כל הסדר חוקי מיוחד על-פני חוק הנאמנות. עם זאת, לגבי כמה סוגי נאמנויות נבחנה מידת התחולה של הוראות חוק הנאמנות לגביהם; שכן בהיותו חוק משלים, חלות לעתים הוראותיו גם על נאמנויות שלכאורה הוסדרו בחוקים מיוחדים.
בנושאי ההגנה בפני הנאמן שאב המחוקק השראה מהמשפט האנגלו - אמרי

In [40]:
# Only the Top Chunks should be given to llm, so we choose chunks with a score of 0.8
top_chunks = [item for item in rerank_results if item["score"] >= 0.6]
print_chunks(top_chunks, True)

Total Chunks:  1
--------
CHUNK 1
--------
Id: 5, Score: 0.69921875
## יג. פרשנות החוק
295 בדרך-כלל ניתן להסתייע בפרשנות חוק בסיסי, דוגמת חוק הנאמנות, בעבודות ההכנה שקדמו לחוק, ובמיוחד בדברי ההסבר להצעת החוק. אולם במקרים רבים לא ניתן היה להסתייע בדברי ההסבר להצעת החוק, גם כשלכאורה ניסוח החוק דומה לנוסח הצעת החוק, כיוון שהחוק שונה מההצעה בתפישותיו היסודיות וכן בפרטים מהותיים.
:
:
-
--
i
.
..
306 סעיף 83.
307 סעיף 72.
89
<!-- PageFooter="nero.co.il" -->
<!-- PageNumber="§29" -->
<!-- PageHeader="מבוא" -->
<!-- PageHeader="פרשנות החוק" -->
בפרשנות החוק הנחנו, שיש לראות את הוראות חוק הנאמנות כמשתלבות, ולכן כמתיישבות, עם הוראות חוקים אחרים, העוסקות באותם נושאים או בנושאים קרובים. הנחה זו מסתברת מסעיף 42 לחוק, המכיר בעדיפותו של כל הסדר חוקי מיוחד על-פני חוק הנאמנות. עם זאת, לגבי כמה סוגי נאמנויות נבחנה מידת התחולה של הוראות חוק הנאמנות לגביהם; שכן בהיותו חוק משלים, חלות לעתים הוראותיו גם על נאמנויות שלכאורה הוסדרו בחוקים מיוחדים.
בנושאי ההגנה בפני הנאמן שאב המחוקק השראה מהמשפט האנגלו - אמריק